## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.

1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).

2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  

3. To generate vectors and split the instances into training and testing datasets at random.

4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.

5. To evaluate the performance of the classifier.


You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [61]:
import a2
from sklearn.svm import LinearSVC



In [62]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [63]:

inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[0:20]

,Line_No,Sentence_No,Word,POS,NER
0,0,1,thousand,NNS,O
1,1,1,of,IN,O
2,2,1,demonstrator,NNS,O
3,3,1,have,VBP,O
4,4,1,march,VBN,O
5,5,1,through,IN,O
6,6,1,london,NNP,B-geo
7,7,1,to,TO,O
8,8,1,protest,VB,O
9,9,1,the,DT,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [64]:

instances = a2.create_instances(inputdata)

In [65]:
instances[20:40]

[Class: gpe Features: ['<S5>', 'this', 'week', 'restart', 'part', 'the', 'conversion', 'process', 'at', '</S5>'],
 Class: geo Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S5>', 'official', 'say', 'they', 'expect', 'get', 'access', 'to', 'sealed', 'sensitive'],
 Class: tim Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'surveillance', 'begin', 'function', '</S5>', '</S4>', '</S3>'],
 Class: org Features: ['<S5>', 'surveillance', 'system', 'begin', 'function', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: org Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'backing', 'threaten', 'to', 'refer', '</S5>', '</S4>'],
 Class: gpe Features: ['<S5>', 'backing', 'have', 'threaten', 'to', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'which', 'could', 'impose', 'sanction', 'if'],
 Class: org Features: ['<S3>', '<S4>', '<S5>', 'which', 'could', 'sanction', 'if',

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [66]:

bigdf = a2.create_table(instances)
bigdf[20:40]

,Class,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,5.032928,-1.120715,0.375585,0.116922,0.145641,-0.487438,-0.001267,-0.004255,0.001975,...,0.000009,-0.000009,-1.015137e-05,0.000007,0.000007,0.000003,-0.000023,-0.000017,-0.000028,-0.000022
21,gpe,5.989315,0.056117,-0.157471,-0.094129,-0.125747,-0.019278,-0.004521,-0.022386,0.034266,...,0.000612,0.000910,-9.111536e-04,0.000999,0.000306,0.001713,-0.000620,-0.001243,-0.000801,-0.002649
22,gpe,4.991326,0.047065,-0.132388,-0.079418,-0.106476,-0.016392,-0.003859,-0.019125,0.029345,...,0.000151,0.000205,-2.200695e-04,0.000254,0.000057,0.000402,-0.000143,-0.000283,-0.000167,-0.000646
23,gpe,5.030752,0.568323,0.758935,0.168941,0.158547,0.022137,-0.021922,0.016067,-0.048622,...,0.000339,0.000669,-2.633947e-05,0.000183,0.000339,-0.000201,-0.000452,0.000070,0.000681,-0.000527
24,gpe,7.997703,0.133758,-0.194957,-0.166327,-0.280392,-0.060328,0.018610,-0.101562,0.285300,...,-0.001183,-0.012904,-7.305000e-03,0.006051,0.000057,0.013404,0.017809,0.005645,0.003731,-0.002070
25,gpe,6.028335,-0.904722,0.224634,0.021182,-0.067715,0.884588,0.006081,0.025655,-0.028096,...,0.000014,-0.000012,-1.641171e-05,0.000012,0.000010,0.000008,-0.000035,-0.000027,-0.000042,-0.000037
26,gpe,6.000123,0.074781,-0.239641,-0.181075,-0.329846,-0.087698,-0.062631,-0.550162,-0.670491,...,0.000006,0.000009,-9.392105e-06,0.000011,0.000002,0.000017,-0.000006,-0.000012,-0.000007,-0.000028
27,gpe,5.998953,0.072186,-0.226844,-0.164635,-0.280608,-0.063379,-0.026210,-0.155031,0.548019,...,0.000007,0.000010,-1.051750e-05,0.000012,0.000002,0.000019,-0.000007,-0.000013,-0.000008,-0.000031
28,gpe,6.012878,0.124895,-0.685803,0.906508,0.261858,0.018889,0.002698,0.011721,-0.011259,...,0.000010,0.000049,6.631323e-05,-0.000010,-0.000027,-0.000048,-0.000019,-0.000101,0.000006,0.000013
29,gpe,6.008186,0.099747,-0.404621,-0.711291,0.869033,0.038004,0.005342,0.023739,-0.027885,...,-0.000027,-0.000020,-8.107272e-06,0.000039,0.000059,0.000002,-0.000040,-0.000028,-0.000014,0.000037


In [67]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

SyntaxError: invalid syntax (<ipython-input-67-072b14b1194a>, line 3)

In [ ]:
len(test_y) / (len(test_y) + len(train_y))

In [ ]:
len(test_X) / (len(test_X) + len(train_X))

In [ ]:
test_y[0]

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [ ]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [ ]:
train_predictions

In [ ]:
train_y

In [ ]:
test_predictions

In [ ]:
test_y

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [ ]:
a2.confusion_matrix(test_y, test_predictions)

In [ ]:
a2.confusion_matrix(train_y, train_predictions)

Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')